In [1]:
import sys

sys.path.append("../")

import sqlite3
import pandas as pd
from functions.env import DB_SCIENCE_PATH_NEW

data_path = "/Users/charlesdedampierre/Desktop/cultura_project/cultura.db"

conn = sqlite3.connect(data_path)
new_conn = sqlite3.connect(DB_SCIENCE_PATH_NEW)
df_ind_birth = pd.read_sql("SELECT * FROM individuals_main_information", conn)

In [2]:
df_ind = pd.read_sql("SELECT * FROM individual_occupations", conn)

only_writers = ['philosopher', 'theologian', 'historian', 'musicologist', 'linguist']
df_ind_writers = df_ind[df_ind["occupations_name"].isin(only_writers)]

df_ind_science = df_ind[df_ind["occupations_category"].str.contains("science")]
list_individuals = list(set(df_ind_science.individual_wikidata_id))

df_ind_science = df_ind_science[df_ind_science['individual_wikidata_id'].isin(list_individuals)]

df_ind = pd.concat([df_ind_science, df_ind_writers])
#df_ind = df_ind.drop_duplicates()
len(set(df_ind.individual_wikidata_id))

71331

In [3]:
df_ind_regions = pd.read_sql("SELECT * FROM individuals_regions", conn)
df_ind_regions = df_ind_regions.drop("individual_name", axis=1)

df_ind_regions = pd.merge(df_ind_regions, df_ind, on = 'individual_wikidata_id')
df_ind_regions = df_ind_regions[
    ["individual_wikidata_id", "individual_name", "region_code", "region_name"]
].drop_duplicates()

print(len(set(df_ind_regions.individual_wikidata_id)))

df_ind_regions.to_sql("individuals_regions", new_conn, if_exists="replace", index=False)

52919


186524

In [4]:

df_ind_birth = df_ind_birth[["individual_wikidata_id", "birthyear"]]
df_ind_birth["productive_year"] = df_ind_birth["birthyear"] + 35

df_ind_final = pd.merge(df_ind, df_ind_birth, on="individual_wikidata_id")

print(len(set(df_ind_final.individual_wikidata_id)))

df_ind_final.to_sql(
    "individuals_occupation_information", new_conn, if_exists="replace", index=False
)


71331


91185

In [5]:
df_ind_final[df_ind_final['birthyear']<=1700].occupations_name.value_counts()

theologian       6305
historian        4400
philosopher      3155
mathematician    1742
explorer         1200
                 ... 
muwaqqit            1
embryologist        1
phycologist         1
gold miner          1
herpetologist       1
Name: occupations_name, Length: 130, dtype: int64